In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from deepctr.models import WDL
from deepctr.feature_column import SparseFeat, get_feature_names
from sklearn.metrics import mean_absolute_error

In [3]:
# 数据加载
df = pd.read_csv('F:/BI/第五周/核心板/L5/libfm/ratings.csv')

In [5]:
# 选取特征
sparse_features = ["userId", "movieId", "timestamp"]
target = ['rating']

In [6]:
for feature in sparse_features:
    transfer = LabelEncoder()
    df[feature] = transfer.fit_transform(df[feature])

In [8]:
fixlen_feature_columns = [SparseFeat(f, df[f].nunique()) for f in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [11]:
# 计算特征gender中不同特征值的个数
df['userId'].nunique()

7120

In [12]:
SparseFeat(name='userId', vocabulary_size=7120, embedding_dim=4)

SparseFeat(name='userId', vocabulary_size=7120, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x0000017496B07088>, embedding_name='userId', group_name='default_group', trainable=True)

In [ ]:
# 数据集拆分

In [13]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=30)

In [14]:
df_train.shape, df_test.shape

((838860, 4), (209715, 4))

In [15]:
train_model_input = {n:df_train[n].values for n in feature_names}
test_model_input = {n:df_test[n].values for n in feature_names}

In [ ]:
# 模型的建立

In [16]:
model = WDL(
    linear_feature_columns,
    dnn_feature_columns,
    task='regression',
    dnn_hidden_units=(256, 256),
    l2_reg_linear=10,
    l2_reg_embedding=10,
    l2_reg_dnn=1,
    dnn_dropout=0.5,
    dnn_activation='relu',
    seed=1020
           )

In [17]:
# 模型编译
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [18]:
# 训练模型
%%time
EPOCH = 10
history = model.fit(
    train_model_input,
    df_train[target].values,
    batch_size=256,
    epochs=EPOCH,
    validation_split=0.2,
    shuffle=True,
    verbose=1,
    workers=2
)

hist_dict  = history.history

Epoch 1/10


F:\Anaconda_download\anaconda_0\lib\site-packages\tensorflow\python\framework\indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


2622/2622 [==============================] - 104s 40ms/step - loss: 5.3455 - mse: 1.3650 - val_loss: 1.2206 - val_mse: 1.1031
Epoch 2/10
2622/2622 [==============================] - 103s 39ms/step - loss: 1.2185 - mse: 1.1685 - val_loss: 1.1721 - val_mse: 1.1037
Epoch 3/10
2622/2622 [==============================] - 103s 39ms/step - loss: 1.1981 - mse: 1.1510 - val_loss: 1.1632 - val_mse: 1.1033
Epoch 4/10
2622/2622 [==============================] - 104s 40ms/step - loss: 1.1833 - mse: 1.1366 - val_loss: 1.1583 - val_mse: 1.1042
Epoch 5/10
2622/2622 [==============================] - 103s 39ms/step - loss: 1.1735 - mse: 1.1275 - val_loss: 1.1567 - val_mse: 1.1023
Epoch 6/10
2622/2622 [==============================] - 103s 39ms/step - loss: 1.1660 - mse: 1.1205 - val_loss: 1.1578 - val_mse: 1.1031
Epoch 7/10
2622/2622 [==============================] - 103s 39ms/step - loss: 1.1615 - mse: 1.1158 - val_loss: 1.1555 - val_mse: 1.1023
Epoch 8/10
2622/2622 [==============================

In [19]:
hist_dict.keys()

dict_keys(['loss', 'mse', 'val_loss', 'val_mse'])

In [20]:
# 预测
test_preds = model.predict(test_model_input, batch_size=256)

In [21]:
test_preds[:2]

array([[3.5383964],
       [3.5379453]], dtype=float32)

In [22]:
# 计算MSE
mean_squared_error(df_test[target].values, test_preds)

1.109423841748677

In [23]:
df_peds = df_test.copy()

In [24]:
df_peds.shape

(209715, 4)

In [25]:
df_peds.head(1)

,userId,movieId,rating,timestamp
817193,5450,1190,3.5,588296


In [26]:
test_preds.shape

(209715, 1)

In [27]:
df_peds['preds'] = test_preds[:, 0]

In [28]:
# 真实值与预测值对比
df_peds[['rating', 'preds']].head()

,rating,preds
817193,3.5,3.538396
438441,2.0,3.537945
370515,5.0,3.537991
840250,4.5,3.538244
250809,3.0,3.538232
